In [2]:
#Quiero hacer cross validation para certificar que el RMSE está bien

import cv2 as cv
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# Leer el archivo CSV con las respuestas correctas
csv_file = "train/train.csv"
df = pd.read_csv(csv_file)

# Crear listas para almacenar las características y las etiquetas
features = []
labels = []

# Procesar cada imagen y extraer las características
images_folder = 'train/train'
for filename in os.listdir(images_folder):
    if filename.endswith('.png'):
        # Leer la imagen
        img_path = os.path.join(images_folder, filename)
        img = cv.imread(img_path)

        # Extraer las características de la imagen (por ejemplo, el número de contornos)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        canny = cv.Canny(gray, 100, 200)
        _, thresh = cv.threshold(canny, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        num_contours = len(contours)

        # Obtener el número real de clips del archivo CSV correspondiente
        image_id = int(filename.split('-')[1].split('.')[0])
        real_clips = df.loc[df['id'] == image_id, 'clip_count'].values[0]

        # Agregar las características y la etiqueta a las listas
        features.append(num_contours)
        labels.append(real_clips)

# Convertir las listas en arrays numpy
features = np.array(features)
labels = np.array(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba una sola vez
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Entrenar un modelo (ejemplo utilizando Random Forest)
model = RandomForestRegressor()

# Realizar validación cruzada con 5 pliegues
scores = cross_val_score(model, X_train.reshape(-1, 1), y_train, cv=5, scoring='neg_mean_squared_error')

# Calcular métricas de evaluación promediando los resultados de los pliegues
mse_cv = -scores.mean()
r2_cv = scores.mean()
print(f"MSE (Cross-validation): {mse_cv}")
print(f"R2 score (Cross-validation): {r2_cv}")
print(f"RMSE (Cross-validation): {np.sqrt(mse_cv)}")

MSE (Cross-validation): 24.3477322033734
R2 score (Cross-validation): -24.3477322033734
RMSE (Cross-validation): 4.934342124678162


In [3]:
scores

array([-24.34225667, -24.53800569, -23.47416519, -25.49867718,
       -23.88555629])